# Corridor experiments

In this notebook, we look at matchings with corridor navground robots. 

First, let us import a few important modules for this task.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)

Next, let us execute the Navground corridor experiment.

In [ ]:
length = 8.0
num_steps = 500
width=1.0
num_agents = 30 # 38
for behaviour in ["ORCA", "HL"]:
    path=f"recorded_experiment_{behaviour}.h5"
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    runs: 1
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 0.12
            horizon: 5.0
            safety_margin: 0.034
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

Reload both experiments and save runs into a dictionary.

In [ ]:
runs = {}
# Reload HL simulation
path_HL = "recorded_experiment_HL.h5"
recorded_experiment = sim.RecordedExperiment(path_HL)
runs["HL"] = recorded_experiment.runs[0]
# Reload ORCA simulation
path_ORCA = "recorded_experiment_ORCA.h5"
recorded_experiment = sim.RecordedExperiment(path_ORCA)
runs["ORCA"] = recorded_experiment.runs[0]

Let us display this simulation in a video

In [ ]:
from navground.sim.ui import WebUI
from navground.sim.notebook import notebook_view
from navground.sim.replay import RealTimeReplay

web_ui = WebUI(host='127.0.0.1', max_rate=-1)
await web_ui.prepare()

First, let us visualise the HL experiment.

In [ ]:
notebook_view(width=300)

In [ ]:
rt_sim = RealTimeReplay(run=runs["HL"], factor=20, web_ui=web_ui)
await rt_sim.run()

Now, we visualise the ORCA experiment.

In [ ]:
rt_sim = RealTimeReplay(run=runs["ORCA"], factor=20, web_ui=web_ui)
await rt_sim.run()

Both simulations are very different. In partiuclar, we observe mainly two dynamics, either the robots end up going in straight trajectories or they get stuck. Both examples below where produced with the same variables.

### Pairwise Matchings

We are now going to compute the induced matchings and their associated diagrams.

Also, we set up the variable "weight" and the timestep shift for our experiments.

In [ ]:
weight = 2 # This is the weight for velocities
shift_time = 30 # Shift between timesteps
steps_list = list(range(0, 400, 5)) # Starting timesteps that we consider

Now, we are going to start by considering two timesteps and their associated divergence diagrams. Notice that these do not change much.

In [ ]:
start_step = 50
gs_kw = dict(width_ratios=[5, 2, 2], height_ratios=[1,1])
fig, axd = plt.subplot_mosaic([['points_HL', 'Diag_HL', 'Diag_ORCA'],
                               ['points_ORCA', 'Diag_HL', 'Diag_ORCA']],
                              gridspec_kw=gs_kw, figsize=(16, 4),
                              layout="constrained")
for behaviour in ["ORCA", "HL"]:
    ps = np.array(runs[behaviour].poses)[:,:,:2]
    twists = np.array(runs[behaviour].twists)[:,:,:2]
    X = ps[start_step]
    Y = ps[start_step + shift_time]
    vel_X = twists[start_step]
    vel_Y = twists[start_step + shift_time]
    X_len = X.shape[0]-1
    # Plot two timesteps
    # fig, ax = plt.subplots(figsize=(8, 1))
    ax = axd[f"points_{behaviour}"]
    perdiver.plot_two_timesteps_with_velocities(X, Y, vel_X, vel_Y, ax, arrow_width=0.03)
    ax.set_title(f"points_{behaviour}", fontsize=20)
    # Plot matching diagram
    ax = axd[f"Diag_{behaviour}"]
    Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
    match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(match_diagram, ax, color="blue")
    ax.set_title(behaviour, fontsize=20)

perdiver.same_diagram_scale(axd["Diag_HL"], axd["Diag_ORCA"])
plt.savefig(f"two_timesteps_points_matching_diags_ORCA_HL.png")

Last, we compute the persistence matching diagram across the step list. There seems to be an equilibrium.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(9,4))
diagrams_behaviour = {}
for j, behaviour in enumerate(["ORCA", "HL"]):
    ps = np.array(runs[behaviour].poses)[:,:,:2]
    twists = np.array(runs[behaviour].twists)[:,:,:2]
    diagrams_list = []
    for idx, start_step in enumerate(steps_list):
        X = ps[start_step]
        Y = ps[start_step + shift_time]
        vel_X = twists[start_step]
        vel_Y = twists[start_step+shift_time]
        Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
        match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
        perdiver.plot_matching_diagram(match_diagram, ax[j], color=mpl.colormaps["GnBu"](idx/len(steps_list)))
        diagrams_list.append(match_diagram)
    # end for 
    diagrams_behaviour[f"{behaviour}"] = diagrams_list
    norm = mpl.colors.Normalize(vmin=steps_list[0], vmax=steps_list[-1])
    cmap = mpl.colormaps["GnBu"]
    mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    plt.colorbar(mappable=mappable, ax=ax[j])
    ax[j].set_title(f"Evolution matching diagram {behaviour}")
# end for
perdiver.same_diagram_scale(ax[0], ax[1])
plt.savefig(f"{plots_dir}/evolution_matching_ORCA_HL.png")

### Vectorisation of matching diagrams

Use persistence images for this.

In [ ]:
from gudhi import representations

In [ ]:
npixels = 15
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.06)
perim.fit(diagrams_behaviour["HL"] + diagrams_behaviour["ORCA"])
perim_arr_dict = {}
perim_arr_dict["HL"] = perim.transform(diagrams_behaviour["HL"])
perim_arr_dict["ORCA"] = perim.transform(diagrams_behaviour["ORCA"])

Visualise two persistence images.

In [ ]:
def timestep_to_idx(steps_list, time):
    return np.argmax((np.array(steps_list) - 20)>=0)

In [ ]:
idx = timestep_to_idx(steps_list, 70)
fig, ax = plt.subplots(ncols=4, figsize=(11,3), layout="constrained")
for idx, behaviour in enumerate(["HL", "ORCA"]):
    ax[2*idx].imshow(perim_arr_dict[behaviour][idx].reshape(15,15))
    ax[2*idx].set_title(behaviour, fontsize=20)
    perdiver.plot_matching_diagram(diagrams_behaviour[behaviour][idx], ax[1 + 2*idx])
    ax[1 + 2*idx].set_aspect("equal")
    ax[1 + 2*idx].set_title(behaviour, fontsize=20)
# end for 
# plt.tight_layout()
plt.savefig(f"{plots_dir}/image_persistence_ORCA_HL.png")

### Divergence vectors across time

Next, we compute and print the divergence array across a few simulation steps.

In [ ]:
divergence_arrays = []
for behaviour in ["ORCA", "HL"]:
    ps = np.array(runs[behaviour].poses)[:,:,:2]
    twists = np.array(runs[behaviour].twists)[:,:,:2]
    divergence_list = []
    Z_barcodes_list = []
    for start_step in steps_list:
        X = ps[start_step]
        Y = ps[start_step + shift_time]
        vel_X = twists[start_step]
        vel_Y = twists[start_step + shift_time]
        Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
        divergence_vector = np.sort(perdiver.compute_divergence_vector(Dist_X, Dist_Y))
        divergence_list.append(divergence_vector)

    divergence_arrays.append( np.array(divergence_list).transpose())

vmax = np.max([np.max(divergence_arr) for divergence_arr in divergence_arrays])
vmin = np.min([np.min(divergence_arr) for divergence_arr in divergence_arrays])
for divergence_arr in divergence_arrays:
    ## Save figure 
    fig, ax = plt.subplots(figsize=(10,2))
    mapable = ax.imshow(divergence_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(steps_list[0], steps_list[-1], 0, X.shape[0]))
    ax.set_title(f"Divergence {behaviour}")
    plt.colorbar(mapable)
    plt.tight_layout()
    plt.savefig(f"{plots_dir}/Divergence_vector_evolution_{behaviour}.png")

### Trajectories analysis

Now, we repeat the same experiment, but, instead of considering timesteps, we consider consecutive trajectories.

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
start_step=300
Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color="blue", print_barcode_n_reps=True)
plt.savefig(f"{plots_dir}/divergence_diag.png")

In [ ]:
X_seq = ps[list(range(start_step, start_step+shift_time+1, 10))]
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_aspect("equal")
perdiver.plot_sequence(X_seq, ax)

In [ ]:
divergence_list = []
Z_barcodes_list = []
for start_step in steps_list:
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    divergence, Z_barcode = perdiver.compute_divergence_vector(Dist_X, Dist_Y, Dist_Z)
    divergence_list.append(divergence)
    Z_barcodes_list.append(Z_barcode)

divergence_arr = np.array(divergence_list).transpose()
Z_barcodes_arr = np.array(Z_barcodes_list).transpose()

Next, we plot divergence vectors over time.

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(Z_barcodes_arr)
mapable = ax.imshow(Z_barcodes_arr, aspect="auto", vmax=vmax, extent=(steps_list[0], steps_list[-1], 0, Z_barcodes_arr.shape[0]))
ax.set_title("corridor")
plt.colorbar(mapable)
plt.tight_layout()
plt.savefig(f"{plots_dir}/corridor_Z_array_hist.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(divergence_arr)
mapable = ax.imshow(divergence_arr, aspect="auto", vmax=vmax, extent=(steps_list[0], steps_list[-1], 0, divergence_arr.shape[0]))
print(mapable)
print(np.max(divergence_arr))
ax.set_title("corridor")
plt.colorbar(mapable)
plt.tight_layout()
plt.savefig(f"{plots_dir}/corridor_divergence_diag_hist.png")

Let us plot the persistence diagram over time for trajectories.

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
for idx, start_step in enumerate(steps_list):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(steps_list)))


In [ ]:
# Persistence Images for corridor experiment

In [ ]:
from importlib import reload
reload(perdiver)

In [ ]:
diags = []
for idx, start_step in enumerate(steps_list):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    diags.append(perdiver.get_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(steps_list))))

# Persistence Images

In [ ]:
length = 8.0
num_steps = 2000
# num_steps = 0
width=1.0
yaml = f"""
steps: {num_steps}
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Corridor
  length: {length}
  width: {width} 
  groups:
    -
      type: thymio
      number: 38
      radius: 0.08
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: 0.166
      behavior:
        type: HL
        optimal_speed: 0.12
        horizon: 5.0
        safety_margin: 0.034
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

In [ ]:
diags = []
shift_step=30
steps_list = list(range(0, 400, shift_step))
for idx, start_step in enumerate(steps_list):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    
    diags.append(perdiver.get_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(steps_list))))

In [ ]:
from gudhi import representations

In [ ]:
shift_step = 10
diag_sum_list = []
list_step = int(100/shift_step)
for start_idx in range(0,len(steps_list), list_step ):
    diag_sum_list.append(np.vstack(diags[start_idx: start_idx+list_step ]))

diag_sum_list = diag_sum_list[:10]

In [ ]:
npixels = 30
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.06)
perim.fit(diag_sum_list)
perim_list = perim.transform(diag_sum_list)
# print(diag_trans.shape)
# diag_shift_list = []
# for diag in diags:
#     diag_trans = np.vstack((diag[:,0], diag[:,1])).transpose()
#     diag_shift_list.append(diag_trans)
    
# perim.fit(diag_shift_list)
# perim_list = perim.transform(diag_shift_list)
for image, diag in zip(perim_list, diag_sum_list):
    fig, ax = plt.subplots(ncols=2, figsize=(4,2))
    image = image.reshape(npixels,-1)
    ax[0].imshow(image)
    ax[1].scatter(diag[:,0], diag[:,1])
    ax[1].set_aspect("equal")